In [9]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [10]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [12]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [19]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Center and adjust zoom
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [28]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
5,5,Dingle,PH,2020-10-12 20:43:06,11.00,122.67,75.96,87,100,3.38
9,9,Almeirim,BR,2020-10-12 20:52:33,-1.52,-52.58,88.50,47,76,3.44
13,13,Aitape,PG,2020-10-12 20:52:33,-3.14,142.35,79.27,84,100,4.54
19,19,Corpus Christi,US,2020-10-12 20:52:28,27.80,-97.40,89.01,66,1,14.99
20,20,Touros,BR,2020-10-12 20:50:48,-5.20,-35.46,80.60,74,40,8.05
21,21,Natal,BR,2020-10-12 20:50:56,-5.79,-35.21,80.60,74,40,8.05
23,23,Hamilton,US,2020-10-12 20:52:17,39.18,-84.53,75.99,69,90,11.41
29,29,Cabo San Lucas,MX,2020-10-12 20:52:35,22.89,-109.91,84.00,74,90,4.70
30,30,Salalah,OM,2020-10-12 20:52:35,17.02,54.09,78.80,78,65,3.36
37,37,Lorengau,PG,2020-10-12 20:52:07,-2.02,147.27,82.11,79,100,7.61


In [29]:
preferred_cities_df.count()

City_ID       186
City          186
Country       186
Date          186
Lat           186
Lng           186
Max Temp      186
Humidity      186
Cloudiness    186
Wind Speed    186
dtype: int64

In [30]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Dingle,PH,75.96,11.00,122.67,
9,Almeirim,BR,88.50,-1.52,-52.58,
13,Aitape,PG,79.27,-3.14,142.35,
19,Corpus Christi,US,89.01,27.80,-97.40,
20,Touros,BR,80.60,-5.20,-35.46,
21,Natal,BR,80.60,-5.79,-35.21,
23,Hamilton,US,75.99,39.18,-84.53,
29,Cabo San Lucas,MX,84.00,22.89,-109.91,
30,Salalah,OM,78.80,17.02,54.09,
37,Lorengau,PG,82.11,-2.02,147.27,


In [31]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [34]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Dingle,PH,75.96,11.00,122.67,Camp Pasica
9,Almeirim,BR,88.50,-1.52,-52.58,Ricácio Hotel
13,Aitape,PG,79.27,-3.14,142.35,
19,Corpus Christi,US,89.01,27.80,-97.40,Holiday Inn Corpus Christi Downtown Marina
20,Touros,BR,80.60,-5.20,-35.46,INN NEW HORIZON
21,Natal,BR,80.60,-5.79,-35.21,Villa Park Hotel | Natal
23,Hamilton,US,75.99,39.18,-84.53,Six Acres Bed & Breakfast
29,Cabo San Lucas,MX,84.00,22.89,-109.91,Hotel Tesoro Los Cabos
30,Salalah,OM,78.80,17.02,54.09,HAMDAN PLAZA HOTEL
37,Lorengau,PG,82.11,-2.02,147.27,Lorengau Harbourside Hotel


In [40]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]